In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException,ElementClickInterceptedException,ElementNotInteractableException,TimeoutException
import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
import xlsxwriter
import time

In [3]:
Names = []
doctor_qualification = []
doctor_specializations = []
expriences = []
profile__descriptioncns = []
clinic_addresses = []                
clinic_timings_sessions = []
clinic_localitys = []
consultation_fees = []
Rating_practos = []

In [4]:
citys = ['Agra', 'Ahmedabad','Allahabad', 'Amravati', 'Amritsar', 'Ambala','Anand', 'Aurangabad', 'Bangalore', 'Bardhaman', 'Bareilly', 'Bharuch','Bhopal', 'Bhubaneswar', 'Bogor' 'Bilaspur', 'Chandigarh', 'Chennai', 'Coimbatore', 'Cuttack', 'Dehradun', 'Delhi', 'Dhanbad', 'Ernakulam', 'Faridabad', 'Ghaziabad', 'Gorakhpur', 'Guntur', 'Gurgaon', 'Guwahati', 'Gwalior','Haridwar','Howrah','Hooghly','Hyderabad','Hubli', 'Indore', 'Jabalpur', 'Jaipur', 'Jalandhar', 'Jamshedpur', 'Jhajjar','Jodhpur', 'Kakinada', 'Kanchipuram', 'Kanpur', 'Khammam','Kolhapur', 'Kolkata', 'Kota', 'Kurnool', 'Lucknow', 'Ludhiana', 'Madurai', 'Mangalore', 'Mathura', 'Meerut', 'Mohali', 'Moradabad', 'Madikeri', 'Mumbai', 'Muzaffarnagar', 'Mysore', 'Nagpur', 'Nashik', 'Navi Mumbai', 'Navsari', 'Nellore', 'Noida', 'Panchkula', 'Patna', 'Puducherry', 'Pune', 'Raipur', 'Rajkot', 'Ranchi', 'Roorkee','Salem', 'Solapur', 'Sonipat','South Goa', 'Srinagar', 'Surat', 'Thane', 'Thiruvananthapuram', 'Thrissur', 'Udaipur', 'Vadodara', 'Varanasi', 'Vellore', 'Vijayawada', 'Visakhapatnam', 'Warangal']
specialitys = ['Dentist','Gynecologist/obstetrician','General surgeon','Dermatologist','Orthopedist','Ear-nose-throat (ent) Specialist','Homoeopath','Ayurveda','Endocrinologist','allergist/immunologist','pediatric immunologist and allergist','pediatrician','sexologist','Anesthesiologists','Cardiologists','radiologist','spine and pain specialist','critical care medicine','emergency & critical care','emergency medicine','internal medicine','family physician','psychiatrist','gastroenterologist','hematologist','nephrologist','Neurologists','oncologist','ophthalmologist','osteopath','pathologist','plastic surgeon','Pulmonologists','Rheumatologists','urologist','sports nutritionist']


for city in citys[:]:
    for speciality in specialitys:
        driver = webdriver.Chrome()
        driver.get("https://www.practo.com/")
        location_input = driver.find_element(By.CSS_SELECTOR, 'input[data-qa-id="omni-searchbox-locality"]')
        keyword_input = driver.find_element(By.CSS_SELECTOR, 'input[data-qa-id="omni-searchbox-keyword"]')
        location_input.clear()
        location_input.send_keys(city)      # Change the city here 
        ot = WebDriverWait(driver, 15)
        suggestion_list_locator = (By.XPATH, "//div[@data-qa-id='omni-suggestions-list'] ")
        ot.until(EC.visibility_of_element_located(suggestion_list_locator))
        time.sleep(2)
        location_input.send_keys(Keys.DOWN)
        time.sleep(2)
        location_input.send_keys(Keys.DOWN)
        time.sleep(2)
        location_input.send_keys(Keys.ENTER)
        keyword_input.clear()
        keyword_input.send_keys(speciality)  # You can replace "Dentist" with your desired search term
        wait = WebDriverWait(driver, 15)
        suggestion_list = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'div[data-qa-id="omni-suggestions-list"]')))
        time.sleep(2)
        keyword_input.send_keys(Keys.DOWN)
        # keyword_input.send_keys(Keys.DOWN)
        # keyword_input.send_keys(Keys.DOWN)
        # time.sleep(2)
        keyword_input.send_keys(Keys.ENTER)
        starting_url = driver.current_url
        
        
        
        while True:
            doctor_links = driver.find_elements(By.XPATH, "//h2[@data-qa-id='doctor_name']/ancestor::a")
            for doctor_link in doctor_links:
                # Get the href attribute of each <a> element
                href_link = doctor_link.get_attribute("href")
                response  = requests.get(href_link)
                soup = BeautifulSoup(response.content, 'html.parser')
                Name = soup.find('h1', attrs={"data-qa-id":"doctor-name"})
                if Name != None:
                    Names.append(Name.text)
                else:
                    Names.append(None)
                
                doctor_qualifications=soup.find('p', attrs={"data-qa-id":"doctor-qualifications"})
                if doctor_qualifications != None:
                    doctor_qualification.append(doctor_qualifications.text)
                else:
                    doctor_qualification.append(None)
                    
                doctor_specialization = soup.find('div', attrs={"data-qa-id":"doctor-specializations"})
                try:
                    if doctor_specialization != None:
                        exprience = [i.text for i in doctor_specialization][-1]
                        doctor_specialization = [i.text for i in doctor_specialization][:-1][0]
                        expriences.append(exprience)
                        doctor_specializations.append(doctor_specialization)
                    else:
                        expriences.append(None)
                        doctor_specializations.append(None)
                except IndexError:
                    doctor_specializations.append(None)
                    pass
                    
                    
                
                profile__descriptioncn= soup.find('div', attrs={"data-qa-id":"doctor-summary"})
                if profile__descriptioncn != None:
                    profile__descriptioncns.append(profile__descriptioncn.text)
                else:
                    profile__descriptioncns.append(None)
                
                clinic_address = soup.find('p', attrs={"data-qa-id":"clinic-address"})
                if clinic_address != None:
                    clinic_addresses.append(clinic_address.text)
                else:
                    clinic_addresses.append(None)
                    
                clinic_timings_session = soup.find('div', attrs={"class":"pure-u-1-3 u-cushion--left"})
                if clinic_timings_session != None:
                    clinic_timings_sessions.append(clinic_timings_session.text)
                else:
                    clinic_timings_sessions.append(None)
                
                clinic_locality = soup.find('h4', attrs={"data-qa-id":"clinic-locality"})
                if clinic_locality != None:
                    clinic_localitys.append(clinic_locality.text)
                else:
                    clinic_localitys.append(None)
                
                consultation_fee = soup.find('span', attrs={"data-qa-id":"consultation_fee"})
                if consultation_fee != None:
                    consultation_fees.append(consultation_fee.text)
                else:
                    consultation_fees.append(None)
                Rating_practo = soup.find('span', attrs={"class":"common__star-rating__value"})
                if Rating_practo != None:
                    Rating_practos.append(Rating_practo.text)
                else:
                    Rating_practos.append(None)
                # Print the href link for each element
                # print("Href link:", href_link)
        
            try:
                # Check if there is a "Next" button to go to the next page
                next_button = driver.find_element(By.XPATH, "//a[@data-qa-id='pagination_next']")
                # next_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//a[@data-qa-id='pagination_next']")))
                if "disabled" in next_button.get_attribute("class"):
                    break  # The "Next" button is disabled, exit the loop
                try:
                    try:
                        next_button.click()
                        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//h2[@data-qa-id='doctor_name']/ancestor::a")))
                    except (ElementNotInteractableException,TimeoutException):
                        break
                except ElementClickInterceptedException:
                    break
        
                # Wait for the page to load before proceeding
                
            except NoSuchElementException:
                break  # No "Next" button found, exit the loop
        # except ElementNotInteractableException:
        #     break
    
    
    # data = pd.DataFrame({'Name':Names,'doctor_qualification':doctor_qualification, "doctor_specializations":doctor_specializations,'expriences':expriences, 'clinic_addresses':clinic_addresses, 'clinic_timings_sessions':clinic_timings_sessions, 'clinic_localitys':clinic_localitys, 'consultation_fees':consultation_fees,'Rating_practos':Rating_practos})
    # data
    
    
    # Close the webdriver
        driver.quit()
    # if city == "Agra" and speciality == "Dentist":
    #     data = pd.DataFrame({'Name':Names,'doctor_qualification':doctor_qualification, "doctor_specializations":doctor_specializations,'expriences':expriences, 'clinic_addresses':clinic_addresses, 'clinic_timings_sessions':clinic_timings_sessions, 'clinic_localitys':clinic_localitys, 'consultation_fees':consultation_fees,'Rating_practos':Rating_practos})
    #     data.to_excel("practo_data.xlsx", index=False)
    # else:
    #     existing_data = pd.read_excel("practo_data.xlsx")
    #     combined_data = pd.concat([existing_data, data], ignore_index=True)
    #     with pd.ExcelWriter("practo_data.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    #         combined_data.to_excel(writer, sheet_name='Sheet1', index=False)

The chromedriver version (122.0.6261.128) detected in PATH at C:\Users\aamri\Downloads\chromedriver.exe might not be compatible with the detected chrome version (123.0.6312.86); currently, chromedriver 123.0.6312.86 is recommended for chrome 123.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (122.0.6261.128) detected in PATH at C:\Users\aamri\Downloads\chromedriver.exe might not be compatible with the detected chrome version (123.0.6312.86); currently, chromedriver 123.0.6312.86 is recommended for chrome 123.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (122.0.6261.128) detected in PATH at C:\Users\aamri\Downloads\chromedriver.exe might not be compatible with the detected chrome version (123.0.6312.86); currently, chromedriver 123.0.6312.86 is recommended for chrome 123.*, so it is advised to delete the driver in PATH and retry
The chromedriver version (122.0.6261.128) detected in PATH at C:\Users\aamri\Download

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.86)
Stacktrace:
	GetHandleVerifier [0x00007FF72A59AD02+56930]
	(No symbol) [0x00007FF72A50F602]
	(No symbol) [0x00007FF72A3C42E5]
	(No symbol) [0x00007FF72A3A1D4C]
	(No symbol) [0x00007FF72A4323F7]
	(No symbol) [0x00007FF72A447891]
	(No symbol) [0x00007FF72A42BA43]
	(No symbol) [0x00007FF72A3FD438]
	(No symbol) [0x00007FF72A3FE4D1]
	GetHandleVerifier [0x00007FF72A916F8D+3711213]
	GetHandleVerifier [0x00007FF72A9704CD+4077101]
	GetHandleVerifier [0x00007FF72A96865F+4044735]
	GetHandleVerifier [0x00007FF72A639736+706710]
	(No symbol) [0x00007FF72A51B8DF]
	(No symbol) [0x00007FF72A516AC4]
	(No symbol) [0x00007FF72A516C1C]
	(No symbol) [0x00007FF72A5068D4]
	BaseThreadInitThunk [0x00007FF9B371257D+29]
	RtlUserThreadStart [0x00007FF9B458AA58+40]


In [23]:
data = pd.DataFrame({'Name':Names,'doctor_qualification':doctor_qualification, "doctor_specializations":doctor_specializations,'expriences':expriences, 'clinic_addresses':clinic_addresses, 'clinic_timings_sessions':clinic_timings_sessions, 'clinic_localitys':clinic_localitys, 'consultation_fees':consultation_fees,'Rating_practos':Rating_practos})
data

,Name,doctor_qualification,doctor_specializations,expriences,clinic_addresses,clinic_timings_sessions,clinic_localitys,consultation_fees,Rating_practos
0,Dr. Pranjali Gupta,"BDS, MDS - Paedodontics And Preventive Dentistry","Dentist, Pediatric Dentist, Cosmetic/Aesthetic...",19 Years Experience Overall (17 years as spec...,"Taj Complex, Landmark: Next to Masjid, Agra","Mon - Wed, Fri - Sat11:00 - 02:0005:00 - 07:00...","Delhi Gate, Agra",₹300,5.0
1,Dr. Sumit Kumar Agarwal,"BDS, MDS - Periodontics","Dentist, Periodontist, Implantologist, Dental ...",14 Years Experience Overall (12 years as spec...,"E-146/3, 32A ., Landmark: Near Radhika Footwea...",Mon - Sat10:00 - 02:3005:00 - 08:30Sun10:00 - ...,"Kamla Nagar, Agra",₹400,5.0
2,Dr. Pawan Dhingra,BDS,Dentist,13 Years Experience Overall,"17/106A, Daresi Number 2, Landmark: Opposite t...",Mon - Sun10:00 - 05:00,"Rawatpara, Agra",₹300,4.5
3,Dr. Manoj Golash,BDS,"Dentist, Cosmetic/Aesthetic Dentist",39 Years Experience Overall (35 years as spec...,"#Au-14, Rashmi Palace, Bye Pass Road., Landma...","Mon - Wed, Fri - Sat10:00 - 02:0005:30 - 08:30...","Kamla Nagar, Agra",₹200,4.0
4,Dr. Mohita Sawhney,"BDS, MDS - Conservative Dentistry & Endodontic...","Endodontist, Dentist",14 Years Experience Overall (11 years as spec...,"UG-4, Shanti Madhuvan Plaza, Delhi Gate Road,...","Mon - Wed, Fri - Sun11:00 - 02:3004:00 - 07:30","Delhi Gate, Agra",₹150,5.0
...,...,...,...,...,...,...,...,...,...
129,Dr. Shamita Singhel,"MBBS, DGO, DNB - Obstetrics & Gynecology","Gynecologist, Obstetrician",27 Years Experience Overall (21 years as spec...,"JC 16 & JC 17 Block, Saltlake City Locality : ...",Thu04:00 - 05:00,"Salt Lake, Kolkata",₹1000,4.0
130,Dr. Gautam Dattasarma,"MBBS, DGO",Gynecologist,35 Years Experience Overall (11 years as spec...,"JC 16 & JC 17 Block, Saltlake City Locality : ...",Sat03:00 - 04:00,"Salt Lake, Kolkata",₹1000,4.0
131,Dr. Anindita Pal,"MBBS, DGO","Gynecologist, Obstetrician",12 Years Experience Overall (11 years as spec...,"LB 10 Eastern Metropolitan Bypass, Chingrighat...",Tue03:00 - 05:00Sun04:00 - 06:00,"Beliaghata, Kolkata",₹600,4.0
132,Dr. Ishba Najam,"MBBS, MS - Obstetrics & Gynaecology","Obstetrician, Gynecologist",10 Years Experience Overall (4 years as speci...,"LB 10 Eastern Metropolitan Bypass, Chingrighat...",Fri12:00 - 02:00,"Beliaghata, Kolkata",₹600,4.0


# If value error run this

In [ ]:
# Create arrays with default values (None) for missing data
import pandas as pd

# Create arrays with default values (None) for missing data
default_value = None

# Ensure that all arrays have the same length
max_length = max(len(Names), len(doctor_qualification), len(doctor_specializations), len(expriences), len(clinic_addresses), len(clinic_timings_sessions), len(clinic_localitys), len(consultation_fees), len(Rating_practos))

Names += [default_value] * (max_length - len(Names))
doctor_qualification += [default_value] * (max_length - len(doctor_qualification))
doctor_specializations += [default_value] * (max_length - len(doctor_specializations))
expriences += [default_value] * (max_length - len(expriences))
clinic_addresses += [default_value] * (max_length - len(clinic_addresses))
clinic_timings_sessions += [default_value] * (max_length - len(clinic_timings_sessions))
clinic_localitys += [default_value] * (max_length - len(clinic_localitys))
consultation_fees += [default_value] * (max_length - len(consultation_fees))
Rating_practos += [default_value] * (max_length - len(Rating_practos))

# Create the DataFrame with arrays of the same length
data = pd.DataFrame({
    'Name': Names,
    'doctor_qualification': doctor_qualification,
    'doctor_specializations': doctor_specializations,
    'expriences': expriences,
    'clinic_addresses': clinic_addresses,
    'clinic_timings_sessions': clinic_timings_sessions,
    'clinic_localitys': clinic_localitys,
    'consultation_fees': consultation_fees,
    'Rating_practos': Rating_practos
})
data

In [ ]:
data.to_excel("practo_data.xlsx")

# For loading into Excel

In [ ]:
existing_data = pd.read_excel("practo_data.xlsx")
combined_data = pd.concat([existing_data, data], ignore_index=True)
with pd.ExcelWriter("practo_data.xlsx", engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    combined_data.to_excel(writer, sheet_name='Sheet1', index=False)

Kolkata-dentist
Kolkata-Gynecologist/obstetrician
Hyderabad-dentist
Hyderabad-Gynecologist/obstetrician
Bangalore-dentist
Bangalore-Gynecologist/obstetrician
Mumbai-dentist
Mumbai-Gynecologist/obstetrician
Gurugram-dentist
Noida-dentist
Delhi-dentist
Chennai-dentist
chennai-Gynecologist/obstetrician

In [ ]:
existing_data = pd.read_excel("practo_data.xlsx")
existing_data = existing_data.drop_duplicates()
existing_data['Name']

In [ ]:
unique_city_list = set(['Agra', 'Ahmedabad', 'Amravati', 'Amritsar', 'Anand', 'Aurangabad', 'Bangalore', 'Bardhaman', 'Bareilly', 'Bhopal', 'Bhubaneswar', 'Bilaspur', 'Chandigarh', 'Chennai', 'Coimbatore', 'Cuttack', 'Dehradun', 'Delhi', 'Dhanbad', 'Ernakulam', 'Faridabad', 'Ghaziabad', 'Gorakhpur', 'Guntur', 'Gurgaon', 'Guwahati', 'Gwalior', 'Hyderabad', 'Indore', 'Jabalpur', 'Jaipur', 'Jalandhar', 'Jamshedpur', 'Jodhpur', 'Kakinada', 'Kanpur', 'Kolhapur', 'Kolkata', 'Kota', 'Kurnool', 'Lucknow', 'Ludhiana', 'Madurai', 'Mangalore', 'Mathura', 'Meerut', 'Mohali', 'Moradabad', 'Mumbai', 'Muzaffarnagar', 'Mysore', 'Nagpur', 'Nashik', 'Navi Mumbai', 'Navsari', 'Nellore', 'Noida', 'Panchkula', 'Patna', 'Puducherry', 'Pune', 'Raipur', 'Rajkot', 'Ranchi', 'Roorkee', 'Solapur', 'Srinagar', 'Surat', 'Thane', 'Thiruvananthapuram', 'Thrissur', 'Udaipur', 'Vadodara', 'Varanasi', 'Vellore', 'Vijayawada', 'Visakhapatnam', 'Warangal'])

# Sort the list if needed
# print(len(unique_city_list))

citys = list(unique_city_list)
print(len(citys))

citys.sort()
print(citys)
# for city in citys[0:]:
#     driver = webdriver.Chrome()
#     driver.get("https://www.practo.com/")
#     location_input = driver.find_element(By.CSS_SELECTOR, 'input[data-qa-id="omni-searchbox-locality"]')
#     location_input.clear()
#     location_input.send_keys(city)      # Change the city here 
#     ot = WebDriverWait(driver, 10)
#     suggestion_list_locator = (By.XPATH, "//div[@data-qa-id='omni-suggestions-list'] ")
#     ot.until(EC.visibility_of_element_located(suggestion_list_locator))
#     location_input.send_keys(Keys.DOWN)
#     # time.sleep(2)
#     location_input.send_keys(Keys.DOWN)
#     time.sleep(2)
#     location_input.send_keys(Keys.ENTER)
#     time.sleep(4)
#     driver.quit()
cities_without_pincode = []
ls = []
geolocator = Nominatim(user_agent="DataBC")
for i in citys:
    location = geolocator.geocode(i)
    ls.append(str(location))
    # print(location)
    # cities_without_pincode = []

for item in ls:
    # Split the address by commas
    parts = item.split(',')
    
    # Check if the last part contains a PIN code (a digit-only part)
    last_part = parts[-2].strip()
    
    if not any(char.isdigit() for char in last_part):
        # No PIN code found in the last part, so consider it a city without a PIN code
        cities_without_pincode.append(parts[0].strip())
    
    # Print the list of cities without a PIN code
for city in cities_without_pincode:
    print(city)







In [ ]:
cities_without_pincode

In [ ]:
from bardapi import BardCookies

cookie_dict = {
    "__Secure-1PSID": "bwh9_VeaAuyCKDx3cGytJZbSuTAEJmcmAHfQ5X3trmQDPbYF6sA-l9Ofs4eSxSWWKguhNA.",
    "__Secure-1PSIDTS":"sidts-CjEB3e41hZxnaBf8-2GkDsr79nd4IN1J4EgFXuuvjVb2UDrBXjxSZtwpdw1aDLEW60l_EAA",
    "__Secure-1PSIDCC":"ACA-OxO5j5FU3syktXzxlEZTss_cY5VdUUKpE7J3ubSUHMnb81vOJZp-RhvoO4bgyyd6Fz6y",
    # Any cookie values you want to pass session object.
}
existing_data = pd.read_excel("practo_data.xlsx")
bard = BardCookies(cookie_dict=cookie_dict)
# bard.get_answer("How is the weather today in seoul?")['content']
ls = []
    
for index, row in existing_data.iterrows():
    result = bard.get_answer(f"give me the mobile number or alternate mobile number of {row['Name']},{row['doctor_qualification']},{row['doctor_specializations']}, in {row['clinic_localitys']}")['content']
    print(result)
    print(index)
    ls.append(result)
    if index%5==0:
        time.sleep(30)

In [ ]:
pincode = []

In [ ]:
# import module 
from geopy.geocoders import Nominatim
import re

pin_code_pattern = r'\b\d{6}\b'
# initialize Nominatim API 
existing_data = pd.read_excel("practo_data.xlsx")
geolocator = Nominatim(user_agent="geoapiExercises")

place = "Sanjay Place Agra"
location = geolocator.geocode(place) 
a = str(location)
# print(a)
state_pattern = r'\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)*\b'
match = re.search(state_pattern, a)

if match:
    state = match.group()
    print(f"State name: {state}")
else:
    print("State name not found in the address.")






# pin_code = re.search(pin_code_pattern, a)
# if pin_code:
#     print(type(pin_code.group()))
# a = str(location)
# print(a.split(",")[-2])
# print(a)
# pincode = []
# start_index = 16712
# for index, row in existing_data.iterrows():
#     try:
#         if index >= start_index:
#             # try:
#             #     geolocator = Nominatim(user_agent="geoapiExercises")
#             # except TimeoutError:
#             #     geolocator = Nominatim(user_agent="geoapiExercises")
#             place = row['clinic_localitys'].replace(",","")
#             location = geolocator.geocode(place) 
#             a = str(location)
#             pin_code = re.search(pin_code_pattern, a)
#             if pin_code:
#                 pincode.append(pin_code.group())
#                 # print(pin_code.group())
#             else:
#                 pincode.append(None)
#         # if location != None:
#         #     pin = a.split(",")[-2]
#         #     pincode.append(pin)
#         # else:
#         #     pincode.append(None)
#     except (AttributeError):
#         pincode.append(None)
#         continue
        
        


In [ ]:
existing_data = pd.read_excel("modified_practo_data.xlsx")
# city  = [ row['clinic_localitys'].split(",")[-1] for index, row in existing_data.iterrows() ]
cities = []
for index, row in existing_data.iterrows():
    cities.append(str(row['clinic_localitys']).split(",")[-1])
data = pd.DataFrame({'cities':cities})
existing_data['Cities'] = data['cities']
existing_data.to_excel("practo_data.xlsx", index=False)
cities

In [ ]:
	
# start_index = 12859
# for i in range(len(pincode)):
#     if i >= start_index:
#         pincode.pop(i)
data = pd.DataFrame({'pincode':pincode})
existing_data = pd.read_excel("practo_data.xlsx")
existing_data['Pincode'] = data['pincode']
existing_data.to_excel("modified_practo_data.xlsx", index=False)
# data.to_csv("pincode1.csv")
data

In [ ]:
import re

data = pd.read_excel("practo_data.xlsx")

landmarks = []


for index, row in existing_data.iterrows():

# for item in data:
    # Use a regular expression to find text following "Landmark:" (case-insensitive)
    match = re.search(r'Landmark:.*', str(row['clinic_addresses']), re.I)
    # match = re.search(r'Landmark:\s+(.*?)(?=(,|\s|$))', str(row['clinic_addresses']), re.I)
    if match:
        landmarks.append(match.group().strip())

# Print the extracted landmarks

data = pd.DataFrame({'landmark':landmarks})
existing_data = pd.read_excel("practo_data.xlsx")
existing_data['Landmark'] = data['landmark']
existing_data.to_excel("modified_practo_data.xlsx", index=False)


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

# Initialize the WebDriver (you need to have ChromeDriver or other WebDriver installed)
driver = webdriver.Chrome()
existing_data = pd.read_excel("practo_data.xlsx")
start_index = 20300
for index, row in existing_data.iterrows():
    if index >= start_index:
        # Open Google
        driver.get("https://www.google.com")
        
        # Find the search bar element by name attribute
        search_bar = driver.find_element(By.NAME, "q")
        
        # Enter a search query
        search_bar.send_keys(f"{row['clinic_localitys']} pincode")
        search_bar.send_keys(Keys.ENTER)
        try:
            element = driver.find_element(By.CLASS_NAME,"IZ6rdc")
            element = element.text
            print(element)
        except NoSuchElementException:
            pincode.append(None)
            continue
        if element:
            pincode.append(element)
        else:
            pincode.append(None)
        
        
        # Wait for a few seconds (you might want to use WebDriverWait for better handling)
        # import time
        # time.sleep(1)
    
    # Close the browser
driver.quit()


In [ ]:
city_state_dict = {
    'Agra': 'Uttar Pradesh',
    'Ahmedabad': 'Gujarat',
    'Amravati': 'Maharashtra',
    'Amritsar': 'Punjab',
    'Anand': 'Gujarat',
    'Aurangabad': 'Maharashtra',
    'Bangalore': 'Karnataka',
    'Bardhaman': 'West Bengal',
    'Bareilly': 'Uttar Pradesh',
    'Bhopal': 'Madhya Pradesh',
    'Bhubaneswar': 'Odisha',
    'Bilaspur': 'Uttar Pradesh',
    'Chandigarh': 'Chandigarh',
    'Chennai': 'Tamil Nadu',
    'Coimbatore': 'Tamil Nadu',
    'Cuttack': 'Odisha',
    'Dehradun': 'Uttarakhand',
    'Delhi': 'Delhi',
    'Dhanbad': 'Jharkhand',
    'Ernakulam': 'Kerala',
    'Faridabad': 'Haryana',
    'Ghaziabad': 'Uttar Pradesh',
    'Gorakhpur': 'Uttar Pradesh',
    'Guntur': 'Andhra Pradesh',
    'Gurgaon': 'Haryana',
    'Guwahati': 'Assam',
    'Gwalior': 'Madhya Pradesh',
    'Hyderabad': 'Telangana',
    'Indore': 'Madhya Pradesh',
    'Jabalpur': 'Madhya Pradesh',
    'Jaipur': 'Rajasthan',
    'Jalandhar': 'Punjab',
    'Jamshedpur': 'Jharkhand',
    'Jodhpur': 'Rajasthan',
    'Kakinada': 'Andhra Pradesh',
    'Kanpur': 'Uttar Pradesh',
    'Kolhapur': 'Maharashtra',
    'Kolkata': 'West Bengal',
    'Kota': 'Rajasthan',
    'Kurnool': 'Andhra Pradesh',
    'Lucknow': 'Uttar Pradesh',
    'Ludhiana': 'Punjab',
    'Madurai': 'Tamil Nadu',
    'Mangalore': 'Karnataka',
    'Mathura': 'Uttar Pradesh',
    'Meerut': 'Uttar Pradesh',
    'Mohali': 'Punjab',
    'Moradabad': 'Uttar Pradesh',
    'Mumbai': 'Maharashtra',
    'Muzaffarnagar': 'Uttar Pradesh',
    'Mysore': 'Karnataka',
    'Nagpur': 'Maharashtra',
    'Nashik': 'Maharashtra',
    'Navi Mumbai': 'Maharashtra',
    'Navsari': 'Gujarat',
    'Nellore': 'Andhra Pradesh',
    'Noida': 'Uttar Pradesh',
    'Panchkula': 'Haryana',
    'Patna': 'Bihar',
    'Puducherry': 'Puducherry',
    'Pune': 'Maharashtra',
    'Raipur': 'Chhattisgarh',
    'Rajkot': 'Gujarat',
    'Ranchi': 'Jharkhand',
    'Roorkee': 'Uttarakhand',
    'Solapur': 'Maharashtra',
    'Srinagar': 'Jammu and Kashmir',
    'Surat': 'Gujarat',
    'Thane': 'Maharashtra',
    'Thiruvananthapuram': 'Kerala',
    'Thrissur': 'Kerala',
    'Udaipur': 'Rajasthan',
    'Vadodara': 'Gujarat',
    'Varanasi': 'Uttar Pradesh',
    'Vellore': 'Tamil Nadu',
    'Vijayawada': 'Andhra Pradesh',
    'Visakhapatnam': 'Andhra Pradesh',
    'Warangal': 'Telangana'
}





existing_data = pd.read_excel("modified_practo_data.xlsx")
state = []
for index, row in existing_data.iterrows():
    try:
        city_name = row['Cities']
        state_name = city_state_dict.get(city_name.strip())
        state.append(state_name)
    except AttributeError:
        state.append(None)
        continue


print(state)
# state = []
# # city = city_state_dict.get('Agra')
# # city
# existing_data = pd.read_excel("modified_practo_data.xlsx")
# for index, row in existing_data.iterrows():
#     city_name = row['Cities']
#     # print(city_name)
#     state.append(city_state_dict.get(city_name))
# state

In [ ]:
state

In [ ]:
data = pd.DataFrame({'state':state})
existing_data = pd.read_excel("modified_practo_data.xlsx")
existing_data['State'] = data['state']
existing_data.to_excel("practo_data.xlsx", index=False)

In [ ]:
existing_data = pd.read_excel("practo_data.xlsx")
# existing_data = existing_data.drop('Unnamed: 0', axis=1)
existing_data

In [ ]:
# Assuming your DataFrame is named 'df'
# existing_data['consultation_fees'] = existing_data['consultation_fees'].fillna(0)
# existing_data['consultation_fees'] = existing_data['consultation_fees'].astype(int)
# existing_data['Landmark'] = existing_data['Landmark'].str.replace('Landmark:', '')
current_date = datetime.date.today()
# existing_data['insert_date'] = current_date
# existing_data = existing_data.drop('Rating_practos', axis=1)
# existing_data.to_excel("practo_data.xlsx", index=False)
# existing_data
# existing_data.fillna('Unknown', inplace=True)
# existing_data
existing_data.drop_duplicates()
existing_data
# data_types = existing_data.dtypes
# print(data_types)

In [ ]:
existing_data.to_csv("practo_data.csv", index=False)
existing_data

In [23]:
df = pd.read_csv("icici_med_uw_tariff_tbl_dump_ 7 (2).csv",low_memory=False)
df

,Id,claim_header_id,particular,bill_cost,tariff_cost,bill_version,date,tariff_particular,category,quantity,...,attributes_11,attributes_12,attributes_13,attributes_14,attributes_15,attributes_16,attributes_17,attributes_18,attributes_19,attributes_20
0,5746930,910202301112345864,Cannulation,50.0,50.0,50.0,27-Sep-2(,Cannulation,package charges~~Packages / Surgery / Procedure,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5746931,910202301112345864,Nebulize (One Time),50.0,50.0,50.0,27-Sep-20,Nebulize (One Time),package charges~~Packages / Surgery / Procedure,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5746932,910202301112345864,C-PAP/ Bi-PAP,1500.0,1500.0,1500.0,28-Sep-20,C-PAP/ Bi-PAP,package charges~~Packages / Surgery / Procedure,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5746933,910202301112345864,Nebulize (One Time),50.0,50.0,50.0,28-Sep-2(,Nebulize (One Time),package charges~~Packages / Surgery / Procedure,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5746934,910202301112345864,Nebulize (One Time),50.0,50.0,50.0,28-Sep-2(,Nebulize (One Time),package charges~~Packages / Surgery / Procedure,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
901263,6798658,611202322085294396,Mrs. Maria Shekar,1800.0,1800.0,1800.0,NaN,NaN,DIALYSIS~~DIALYSIS,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
901264,6798659,611202322085294396,Mrs. Maria Shekar,1800.0,1800.0,1800.0,NaN,NaN,DIALYSIS~~DIALYSIS,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
901265,6798660,611202322085294396,Mrs. Maria Shekar,1800.0,1800.0,1800.0,NaN,NaN,DIALYSIS~~DIALYSIS,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
901266,6798661,611202322085294396,Mrs. Maria Shekar,0.0,0.0,0.0,NaN,NaN,DIALYSIS~~DIALYSIS,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
df1 = pd.read_csv("labrayte1.csv")
df

FileNotFoundError: [Errno 2] No such file or directory: 'labrayte1.csv'

In [26]:
city_state_dict = {
    'Agra': 'Uttar Pradesh',
    'Ahmedabad': 'Gujarat',
    'Amravati': 'Maharashtra',
    'Amritsar': 'Punjab',
    'Anand': 'Gujarat',
    'Aurangabad': 'Maharashtra',
    'Bangalore': 'Karnataka',
    'Bardhaman': 'West Bengal',
    'Bareilly': 'Uttar Pradesh',
    'Bhopal': 'Madhya Pradesh',
    'Bhubaneswar': 'Odisha',
    'Bilaspur': 'Uttar Pradesh',
    'Chandigarh': 'Chandigarh',
    'Chennai': 'Tamil Nadu',
    'Coimbatore': 'Tamil Nadu',
    'Cuttack': 'Odisha',
    'Dehradun': 'Uttarakhand',
    'Delhi': 'Delhi',
    'Dhanbad': 'Jharkhand',
    'Ernakulam': 'Kerala',
    'Faridabad': 'Haryana',
    'Ghaziabad': 'Uttar Pradesh',
    'Gorakhpur': 'Uttar Pradesh',
    'Guntur': 'Andhra Pradesh',
    'Gurgaon': 'Haryana',
    'Guwahati': 'Assam',
    'Gwalior': 'Madhya Pradesh',
    'Hyderabad': 'Telangana',
    'Indore': 'Madhya Pradesh',
    'Jabalpur': 'Madhya Pradesh',
    'Jaipur': 'Rajasthan',
    'Jalandhar': 'Punjab',
    'Jamshedpur': 'Jharkhand',
    'Jodhpur': 'Rajasthan',
    'Kakinada': 'Andhra Pradesh',
    'Kanpur': 'Uttar Pradesh',
    'Kolhapur': 'Maharashtra',
    'Kolkata': 'West Bengal',
    'Kota': 'Rajasthan',
    'Kurnool': 'Andhra Pradesh',
    'Lucknow': 'Uttar Pradesh',
    'Ludhiana': 'Punjab',
    'Madurai': 'Tamil Nadu',
    'Mangalore': 'Karnataka',
    'Mathura': 'Uttar Pradesh',
    'Meerut': 'Uttar Pradesh',
    'Mohali': 'Punjab',
    'Moradabad': 'Uttar Pradesh',
    'Mumbai': 'Maharashtra',
    'Muzaffarnagar': 'Uttar Pradesh',
    'Mysore': 'Karnataka',
    'Nagpur': 'Maharashtra',
    'Nashik': 'Maharashtra',
    'Navi Mumbai': 'Maharashtra',
    'Navsari': 'Gujarat',
    'Nellore': 'Andhra Pradesh',
    'Noida': 'Uttar Pradesh',
    'Panchkula': 'Haryana',
    'Patna': 'Bihar',
    'Puducherry': 'Puducherry',
    'Pune': 'Maharashtra',
    'Raipur': 'Chhattisgarh',
    'Rajkot': 'Gujarat',
    'Ranchi': 'Jharkhand',
    'Roorkee': 'Uttarakhand',
    'Solapur': 'Maharashtra',
    'Srinagar': 'Jammu and Kashmir',
    'Surat': 'Gujarat',
    'Thane': 'Maharashtra',
    'Thiruvananthapuram': 'Kerala',
    'Thrissur': 'Kerala',
    'Udaipur': 'Rajasthan',
    'Vadodara': 'Gujarat',
    'Varanasi': 'Uttar Pradesh',
    'Vellore': 'Tamil Nadu',
    'Vijayawada': 'Andhra Pradesh',
    'Visakhapatnam': 'Andhra Pradesh',
    'Warangal': 'Telangana'
}





existing_data = pd.read_csv("labraytes1.csv")
state = []
for index, row in existing_data.iterrows():
    try:
        city_name = row['Cities']
        state_name = city_state_dict.get(city_name.strip())
        state.append(state_name)
    except AttributeError:
        state.append(None)
        continue


print(state)
# state = []
# # city = city_state_dict.get('Agra')
# # city
# existing_data = pd.read_excel("modified_practo_data.xlsx")
# for index, row in existing_data.iterrows():
#     city_name = row['Cities']
#     # print(city_name)
#     state.append(city_state_dict.get(city_name))
# state

['Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Pradesh', 'Uttar Prades

In [27]:
data = pd.DataFrame({'state':state})
existing_data = pd.read_csv("labraytes1.csv")
existing_data['State'] = data['state']
existing_data.to_csv("labrayte1.csv", index=False)

In [2]:
citys = ['Agra', 'Ahmedabad', 'Amravati', 'Amritsar', 'Anand', 'Aurangabad', 'Bangalore', 'Bardhaman', 'Bareilly', 'Bhopal', 'Bhubaneswar', 'Bilaspur', 'Chandigarh', 'Chennai', 'Coimbatore', 'Cuttack', 'Dehradun', 'Delhi', 'Dhanbad', 'Ernakulam', 'Faridabad', 'Ghaziabad', 'Gorakhpur', 'Guntur', 'Gurgaon', 'Guwahati', 'Gwalior', 'Hyderabad', 'Indore', 'Jabalpur', 'Jaipur', 'Jalandhar', 'Jamshedpur', 'Jodhpur', 'Kakinada', 'Kanpur', 'Kolhapur', 'Kolkata', 'Kota', 'Kurnool', 'Lucknow', 'Ludhiana', 'Madurai', 'Mangalore', 'Mathura', 'Meerut', 'Mohali', 'Moradabad', 'Mumbai', 'Muzaffarnagar', 'Mysore', 'Nagpur', 'Nashik', 'Navi Mumbai', 'Navsari', 'Nellore', 'Noida', 'Panchkula', 'Patna', 'Puducherry', 'Pune', 'Raipur', 'Rajkot', 'Ranchi', 'Roorkee', 'Solapur', 'Srinagar', 'Surat', 'Thane', 'Thiruvananthapuram', 'Thrissur', 'Udaipur', 'Vadodara', 'Varanasi', 'Vellore', 'Vijayawada', 'Visakhapatnam', 'Warangal']
len(citys)

78

In [1]:
ab = ['Agra', 'Ahmedabad','Allahabad', 'Amravati', 'Amritsar', 'Ambala','Anand', 'Aurangabad', 'Bangalore', 'Bardhaman', 'Bareilly', 'Bharuch','Bhopal', 'Bhubaneswar', 'Bogor' 'Bilaspur', 'Chandigarh', 'Chennai', 'Coimbatore', 'Cuttack', 'Dehradun', 'Delhi', 'Dhanbad', 'Ernakulam', 'Faridabad', 'Ghaziabad', 'Gorakhpur', 'Guntur', 'Gurgaon', 'Guwahati', 'Gwalior','Haridwar','Howrah','Hooghly','Hyderabad','Hubli', 'Indore', 'Jabalpur', 'Jaipur', 'Jalandhar', 'Jamshedpur', 'Jhajjar','Jodhpur', 'Kakinada', 'Kanchipuram', 'Kanpur', 'Khammam','Kolhapur', 'Kolkata', 'Kota', 'Kurnool', 'Lucknow', 'Ludhiana', 'Madurai', 'Mangalore', 'Mathura', 'Meerut', 'Mohali', 'Moradabad', 'Madikeri', 'Mumbai', 'Muzaffarnagar', 'Mysore', 'Nagpur', 'Nashik', 'Navi Mumbai', 'Navsari', 'Nellore', 'Noida', 'Panchkula', 'Patna', 'Puducherry', 'Pune', 'Raipur', 'Rajkot', 'Ranchi', 'Roorkee','Salem', 'Solapur', 'Sonipat','South Goa', 'Srinagar', 'Surat', 'Thane', 'Thiruvananthapuram', 'Thrissur', 'Udaipur', 'Vadodara', 'Varanasi', 'Vellore', 'Vijayawada', 'Visakhapatnam', 'Warangal']
len(ab)

92

In [3]:
specialitys = ['Dentist','Gynecologist/obstetrician','General surgeon','Dermatologist','Orthopedist','Ear-nose-throat (ent) Specialist','Homoeopath','Ayurveda','Endocrinologist','allergist/immunologist','pediatric immunologist and allergist','pediatrician','sexologist','Anesthesiologists','Cardiologists','radiologist','spine and pain specialist','critical care medicine','emergency & critical care','emergency medicine','internal medicine','family physician','psychiatrist','gastroenterologist','hematologist','nephrologist','Neurologists','oncologist','ophthalmologist','osteopath','pathologist','plastic surgeon','Pulmonologists','Rheumatologists','urologist','sports nutritionist']
len(specialitys)

36

In [4]:
specialitys = ['Dentist','Gynecologist/obstetrician','General surgeon','Dermatologist','Orthopedist','Ear-nose-throat (ent) Specialist','Homoeopath','Ayurveda','Endocrinologist',]
len(specialitys)

9